<a href="https://colab.research.google.com/github/botmaker00/Rex-Deploy/blob/main/rex_hk_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align:center;">🚀 REX ENCODER BOT – HEROKU DEPLOY</h1>
<div align="center">
<img src="https://files.catbox.moe/4w80av.jpg" height="220" style="border-radius:15px; box-shadow:0 4px 18px rgba(0,0,0,.35);"><br><br>
<b>Channel:</b> @RexBots_Official &nbsp;|&nbsp; <b>Support:</b> @rexbotschat &nbsp;|&nbsp; <b>Repo:</b> <a href="https://github.com/abhinai2244/Encoding-Bot">Encoding-Bot</a>
</div>

---
🇬🇧 **What this notebook does (English):** Easy one‑by‑one Heroku deploy for your Encoder Bot. You can set config in 3 ways and deploy multiple apps in one go. Each step has a short explanation.

🇮🇳 **Ye notebook kya karta hai (Hindi):** Isse tum apna Encoder Bot Heroku pe bahut easily deploy kar sakte ho. Config 3 tareekon se set kar sakte ho, aur ek se zyada apps bhi deploy ho sakti hain. Har step ke upar chhota explain hai.


### 🔍 Quick Flow
1️⃣ Heroku Login – email + API key (must)

2️⃣ Create Apps (optional) – new Heroku apps if needed

3️⃣ Config Setup – Manual / Gist URL / Upload file (ONLY ONE active)

4️⃣ Deploy – push all configured apps

5️⃣ Logs – check errors & status

6️⃣ Logout – optional, for safety

➡️ **Tip:** Just run cells from top to bottom. Click ▶ on each cell.


In [ ]:
# ==============================
# 1️⃣ Heroku Login (Required)
# ==============================
# EN: Put your Heroku account email and API key here.
#    Without this, deploy will NOT work.
# HI: Yahan Heroku ka email aur API key daalo,
#    warna deploy nahi chalega.

Heroku_Email = ""  # @param {type:"string"}
Heroku_API = ""    # @param {type:"string"}

!curl -s https://cli-assets.heroku.com/install.sh | sh
from IPython.display import clear_output, HTML
clear_output()
print("✨ Heroku CLI Installed")

if not Heroku_Email or not Heroku_API:
  raise Exception("❌ Fill Heroku Email & API Key!")

import os
netrc = (
  f"machine api.heroku.com\n login {Heroku_Email}\n password {Heroku_API}\n"
  f"machine git.heroku.com\n login {Heroku_Email}\n password {Heroku_API}"
)
with open(os.path.expanduser("~/.netrc"), "w") as f:
  f.write(netrc)
os.chmod(os.path.expanduser("~/.netrc"), 0o600)

!git config --global user.email {Heroku_Email}
!git config --global user.name "RexBot"

display(HTML("""
<div style='padding:8px 12px; border-radius:8px; background:#e8f5e9; color:#1b5e20;'>
✅ <b>Heroku Login Successful!</b><br>
<span style='font-size:13px;'>You are now ready to create apps and deploy.</span>
</div>
"""))


💡 **English:** Get your Heroku API key from: *Account → Account Settings → API Key* and paste it above.

💡 **Hindi:** Heroku API key nikalne ke liye: *Account → Account Settings → API Key* pe jao aur upar paste karo. Kisi ke saath share mat karo. 🔒


In [ ]:
# =====================================
# 2️⃣ Create Heroku Apps (Optional)
# =====================================
# EN: If you want to create new Heroku apps, write their names here.
#     You can create multiple apps separated by spaces.
# HI: Agar nayi Heroku apps banana hai, yahan unke naam likho.
#     Ek se zyada naam space se alag karke likh sakte ho.

App_Names = ""  # @param {type:"string"}
Region = "eu"   # @param ["eu", "us"]
Team = ""       # @param {type:"string"}

team_flag = f"--team {Team}" if Team else ""
for name in App_Names.split():
  clean = name.strip()
  if clean:
    !heroku create --region {Region} --stack container {team_flag} {clean}


✅ **Example (English):** `rex-encoder-1 rex-encoder-2`

✅ **Example (Hindi):** Agar do bots chahiye, to `rex-bot-a rex-bot-b` likh do.

If your apps are already created in the Heroku dashboard, you can skip this cell. ⏭️


In [ ]:
# ======================================================
# 3️⃣ Rex Bots Config Setup (Choose ONE method)
# ======================================================
# EN: App_Name should be the Heroku app name where you want to deploy.
# HI: App_Name me wahi Heroku app ka naam daalo jisme bot deploy karna hai.

App_Name = ""  # @param {type:"string"}

# ---------- Choose Config Method ----------
# EN: Select how you want to provide config.
# HI: Config dene ka tareeka yahan select karo.

Config_Method = "Manual"  # @param ["Manual", "Gist URL", "Upload File"]

import os, shutil

# Clean old folder if exists
if App_Name and os.path.exists(App_Name):
  shutil.rmtree(App_Name)

!git clone https://github.com/botmaker00/Rex-Deploy {App_Name}
%cd {App_Name}

# remove extra files
for f in ["README.md", "*.ipynb"]:
  !rm -f {f}

if Config_Method == "Manual":
  # ---------- Method 1: Manual Fill ----------
  # EN: Fill all Telegram / DB details here.
  # HI: Yahan sab details manually daalni hain.

  API_ID = 12345678          # @param {type:"integer"}
  API_HASH = ""             # @param {type:"string"}
  BOT_TOKEN = ""            # @param {type:"string"}
  OWNER_ID = 1234567890      # @param {type:"integer"}
  SUDO_USERS = ""           # @param {type:"string"}
  EVERYONE_CHATS = ""       # @param {type:"string"}
  LOG_CHANNEL = -1001234567890  # @param {type:"integer"}
  SESSION_NAME = "encoderbot"  # @param {type:"string"}
  MONGO_URI = ""            # @param {type:"string"}
  UPSTREAM_REPO = ""        # @param {type:"string"}
  UPSTREAM_BRANCH = "main"  # @param {type:"string"}
  DOWNLOAD_DIR = "VideoEncoder/downloads/"  # @param {type:"string"}
  ENCODE_DIR = "VideoEncoder/encodes/"     # @param {type:"string"}
  INDEX_URL = ""            # @param {type:"string"}
  DRIVE_DIR = ""            # @param {type:"string"}

  # build config.env from manual values
  config = f"""# Basics #
API_ID = {API_ID}
API_HASH = "{API_HASH}"
BOT_TOKEN = "{BOT_TOKEN}"

# Authorization #
OWNER_ID = {OWNER_ID}
SUDO_USERS = "{SUDO_USERS}"
EVERYONE_CHATS = "{EVERYONE_CHATS}"

# Log Channel #
LOG_CHANNEL = {LOG_CHANNEL}

# Database #
SESSION_NAME = "{SESSION_NAME}"
MONGO_URI = "{MONGO_URI}"
UPSTREAM_REPO = "{UPSTREAM_REPO}"
UPSTREAM_BRANCH = "{UPSTREAM_BRANCH}"

# Optional #
DOWNLOAD_DIR = "{DOWNLOAD_DIR}"
ENCODE_DIR = "{ENCODE_DIR}"
INDEX_URL = "{INDEX_URL}"
DRIVE_DIR = "{DRIVE_DIR}"
"""

  with open("config.env", "w") as f:
    f.write(config)
  print("📝 config.env created from MANUAL input")

elif Config_Method == "Gist URL":
  # ---------- Method 2: Gist URL (raw) ----------
  # EN: Paste RAW link of config.env from GitHub Gist.
  # HI: GitHub Gist pe jo config.env hai uska RAW link yahan paste karo.

  Gist_URL = ""  # @param {type:"string"}

  if not Gist_URL.strip():
    raise Exception("❌ Gist URL khaali hai! Raw link daalo.")

  !curl -L -o config.env "{Gist_URL}"
  print("🌐 Downloaded config.env from Gist")

elif Config_Method == "Upload File":
  # ---------- Method 3: Upload config.env ----------
  # EN: If you already have config.env file, upload it.
  # HI: Agar config.env file ready hai, to upload karo.

  from google.colab import files
  uploaded = files.upload()
  found = False
  for fn in uploaded.keys():
    os.rename(fn, "config.env")
    found = True
    break
  if not found:
    raise Exception("❌ Koi file upload nahi hui!")
  print("📁 Uploaded config.env")

else:
  raise Exception("❌ Invalid Config_Method selected!")

%cd ..
print("\n📂 Configured Apps (folders):")
!ls -d */


⭐ **English:**
- Select **Manual** if you don’t have any config file yet.
- Select **Gist URL** if your `config.env` is stored on GitHub Gist.
- Select **Upload File** if you want to upload a local `config.env` file from your device.

⭐ **Hindi:**
- **Manual** tab select karo jab config.env ready na ho, sab yahin bharo.
- **Gist URL** tab select karo jab config.env Gist pe save ho.
- **Upload File** tab select karo jab config.env file already bani hui ho tumhare paas.


In [ ]:
# ===============================
# 4️⃣ Deploy All Configured Apps
# ===============================
# EN: Write the app folder names you configured above.
# HI: Upar jo folders config kiye the, unka naam yahan likho.

Deploy_List = ""  # @param {type:"string"}

import os

for app in Deploy_List.split():
  app = app.strip()
  if os.path.isdir(app):
    %cd {app}
    !git add . -f
    !git commit -m "Rex Bots Deploy" --allow-empty
    !heroku git:remote -a {app}
    !git push heroku main -f
    print(f"✅ {app} → Deployed Successfully!")
    %cd ..
  else:
    print(f"⚠️ {app} folder not found!")


📌 **Example:**
- English: If you configured `rex-encoder-1` and `rex-encoder-2`, then write: `rex-encoder-1 rex-encoder-2`.
- Hindi: Agar tumne `rex-encoder-1` aur `rex-encoder-2` setup kiye the, to yahan `rex-encoder-1 rex-encoder-2` likho.


In [ ]:
# ==================
# 5️⃣ Check Logs
# ==================
# EN: Put your Heroku app name to see live logs.
# HI: Heroku app ka naam do, live logs dekhne ke liye.

App = ""  # @param {type:"string"}
!heroku logs -t -a {App}


🔎 **Use this when:**
- English: Bot is not starting, or you see application error.
- Hindi: Bot online nahi aa raha, ya Heroku pe error aa rahi ho.


In [ ]:
# ==================
# 6️⃣ Logout (Optional)
# ==================
# EN: Logout from Heroku when you are done.
# HI: Kaam ho jaaye to Heroku se logout kar sakte ho.

!heroku logout


<div align="center" style="margin-top:15px;">
<b>🎉 All done!</b><br>
If you face any issue, ping <b>@rexbotschat</b> on Telegram. 💬
</div>
